# Working with Emoji Package

In [ ]:
# !pip install emoji

In [ ]:
import emoji
emoji.emojize(":fire:")

In [ ]:
emoji.EMOJI_ALIAS_UNICODE

In [ ]:
emoji_dict={
    '0':':smiling_face_with_heart-eyes:',
    '1':':baseball:',
    '2':':grinning_face_with_big_eyes:',
    '3':':face_with_steam_from_nose:',
    '4':':fork_and_knife:',
    '5':':nerd_face:',
    '6':':smiling_face_with_sunglasses:',
    '7':':beating_heart:',
    '8':':persevering_face:',
    '9':':flushed_face:'
}

# Preprocess Data

In [ ]:
import pandas as pd

train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_emoji.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X_train = train['0']
Y_train = train['1']

X_test = test['0']
Y_test = test['1']

In [ ]:
#One_hot_encoded Y_Train is mandatory ,so...
from keras.utils import to_categorical
Y_train =to_categorical(Y_train,num_classes=10)
Y_test=to_categorical(Y_test,num_classes=10)
print(Y_train[0].shape)
print(Y_train[0])

In [ ]:
# for i in range(150):
#     print(X_train[i],emoji.emojize(emoji_dict[str(Y_train[i])]))

# Glove Vector using 6B.50D.txt

In [ ]:
f=open('glove.6B.50d.txt',encoding='utf-8')

In [ ]:
import numpy as np
embedding_index={}

for lines in f:
    values=lines.split()
    word=values[0]
    coeff=np.asarray(values[1:],dtype='float')
    embedding_index[word]=coeff
f.close()

In [ ]:
embedding_index['i']

# Glove Embeddings (Convert Sentences into Vectors)


In [ ]:
def embedding_output(X):
    maxlen=10
    emb_dim=50#given embedding index has shape of 50
    embedding_out = np.zeros((X.shape[0],maxlen,emb_dim))
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for ij in range(len(X[ix])):
            #go to every word in the current(ix) sentence
            try:
                embedding_out[ix][ij]=embedding_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out
embedding_matrix_train =embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

In [ ]:
print(X_train[0])
print(len(X_train[0]))

In [ ]:
embedding_matrix_train.shape

In [ ]:
embedding_matrix_test.shape

# **Make a RNN / LSTM MODEL**

In [ ]:
from keras.models import Sequential
from keras.layers import *

In [ ]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
checkpoint =ModelCheckpoint('best_model.h5',monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist =model.fit(embedding_matrix_train,Y_train,epochs=200,batch_size=64,shuffle=True,validation_split=0.3,callbacks=[checkpoint,earlystop])

In [ ]:
model.load_weights('best_model.h5')
model.evaluate(embedding_matrix_test,Y_test)

In [ ]:
pred=model.predict_classes(embedding_matrix_test)
pred

In [ ]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dict[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dict[str(pred[i])]))